In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
import math
from matplotlib.pyplot import figure
import scipy.stats as stats
from sklearn import preprocessing


In [2]:
df=pd.read_csv("telco.xls")

In [3]:
df['TotalCharges'] = df["TotalCharges"].replace(" ",np.nan)

In [4]:
def missing_data_function(frame):
        total = frame.isnull().sum().sort_values(ascending=False)
        percent = (frame.isnull().sum()*100 / frame.isnull().count()).sort_values(ascending=False)
        missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        return missing_data

In [5]:
missing=missing_data_function(df)
missing[missing['Total']>0]

,Total,Percent
TotalCharges,11,0.156183


In [6]:
df = df[df["TotalCharges"].notnull()]

In [8]:
df['SeniorCitizen']=df['SeniorCitizen'].astype(object)
df["TotalCharges"]=df["TotalCharges"].astype(float)

#### strategy one 

1- One hot encoding for categorical variables with number of labels > 2 \\
2- Eliminate gender and phone serves 
3- standarization min - max 
4 - SVM 
5 - features importance 

In [9]:
df.dtypes

customerID           object
gender               object
SeniorCitizen        object
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges        float64
Churn                object
dtype: object

In [10]:
data=df.copy()

In [11]:
data=data.drop(["gender","PhoneService"],axis=1)

In [12]:
numericals=data.drop(["customerID","Churn"],axis=1).select_dtypes(exclude='object').keys()
numericals

Index(['tenure', 'MonthlyCharges', 'TotalCharges'], dtype='object')

In [13]:
cat=data.drop(["customerID","Churn"],axis=1).select_dtypes(include='object').keys()
cat

Index(['SeniorCitizen', 'Partner', 'Dependents', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod'],
      dtype='object')

In [14]:
binary=data.drop(["customerID","Churn"],axis=1).nunique()[data.drop(["customerID","Churn"],axis=1).nunique() == 2].keys().tolist()

In [24]:
binary

['SeniorCitizen', 'Partner', 'Dependents', 'PaperlessBilling']

In [15]:
multi_cot = [i for i in cat if i not in binary]
multi_cot

['MultipleLines',
 'InternetService',
 'OnlineSecurity',
 'OnlineBackup',
 'DeviceProtection',
 'TechSupport',
 'StreamingTV',
 'StreamingMovies',
 'Contract',
 'PaymentMethod']

In [16]:
data.drop(["customerID","Churn"],axis=1).select_dtypes(include='object').nunique()

SeniorCitizen       2
Partner             2
Dependents          2
MultipleLines       3
InternetService     3
OnlineSecurity      3
OnlineBackup        3
DeviceProtection    3
TechSupport         3
StreamingTV         3
StreamingMovies     3
Contract            3
PaperlessBilling    2
PaymentMethod       4
dtype: int64

In [17]:
le = preprocessing.LabelEncoder()

In [18]:
for i in binary:
    data[i] = le.fit_transform(data[i])

In [19]:
data.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,0,1,0,1,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,1,Electronic check,29.85,29.85,No
1,5575-GNVDE,0,0,0,34,No,DSL,Yes,No,Yes,No,No,No,One year,0,Mailed check,56.95,1889.50,No
2,3668-QPYBK,0,0,0,2,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,1,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,0,0,0,45,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,0,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,0,0,0,2,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.70,151.65,Yes


In [37]:
data = pd.get_dummies(data = data,columns = multi_cot)

In [38]:
data.head()

,customerID,SeniorCitizen,Partner,Dependents,tenure,PaperlessBilling,Churn,MonthlyCharges,TotalCharges,MultipleLines_No,...,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,0,1,0,1,1,No,-1.161694,-0.994194,0,...,1,0,0,1,0,0,0,0,1,0
1,5575-GNVDE,0,0,0,34,0,No,-0.260878,-0.173740,1,...,1,0,0,0,1,0,0,0,0,1
2,3668-QPYBK,0,0,0,2,1,Yes,-0.363923,-0.959649,1,...,1,0,0,1,0,0,0,0,0,1
3,7795-CFOCW,0,0,0,45,0,No,-0.747850,-0.195248,0,...,1,0,0,0,1,0,1,0,0,0
4,9237-HQITU,0,0,0,2,1,Yes,0.196178,-0.940457,1,...,1,0,0,1,0,0,0,0,1,0


In [53]:
std = preprocessing.StandardScaler()
scaled = std.fit_transform(data[numericals])
scaled = pd.DataFrame(scaled,columns=numericals)

In [54]:
scaled.head(n=3)

,tenure,MonthlyCharges,TotalCharges
0,-1.280248,-1.161509,-0.994437
1,0.064303,-0.260626,-0.174153
2,-1.239504,-0.363679,-0.959899


In [55]:
data = data.drop(columns = numericals,axis = 1)
data = data.merge(scaled,left_index=True,right_index=True,how = "left")

In [78]:
missing=missing_data_function(data)
missing[missing['Total']>0]

,Total,Percent


In [77]:
data=data.dropna(axis=0)

In [79]:
X=data.drop(["customerID","Churn"],axis=1)

In [80]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
SeniorCitizen,7010.0,0.162197,0.368658,0.000000,0.000000,0.000000,0.000000,1.000000
Partner,7010.0,0.482882,0.499743,0.000000,0.000000,0.000000,1.000000,1.000000
Dependents,7010.0,0.299001,0.457853,0.000000,0.000000,0.000000,1.000000,1.000000
PaperlessBilling,7010.0,0.592011,0.491496,0.000000,0.000000,1.000000,1.000000,1.000000
MultipleLines_No,7010.0,0.481598,0.499697,0.000000,0.000000,0.000000,1.000000,1.000000
MultipleLines_No phone service,7010.0,0.096576,0.295401,0.000000,0.000000,0.000000,0.000000,1.000000
MultipleLines_Yes,7010.0,0.421826,0.493886,0.000000,0.000000,0.000000,1.000000,1.000000
InternetService_DSL,7010.0,0.343367,0.474866,0.000000,0.000000,0.000000,1.000000,1.000000
InternetService_Fiber optic,7010.0,0.440086,0.496433,0.000000,0.000000,0.000000,1.000000,1.000000
InternetService_No,7010.0,0.216548,0.411921,0.000000,0.000000,0.000000,0.000000,1.000000


In [81]:
data.Churn.unique()

array(['No', 'Yes'], dtype=object)

In [82]:
data.Churn.value_counts()

No     5146
Yes    1864
Name: Churn, dtype: int64

In [83]:
Y=data["Churn"].map({"No":0,"Yes":1})

In [62]:
Y.unique()

array([0, 1])

In [63]:
Y.value_counts(normalize=True)

0    0.734215
1    0.265785
Name: Churn, dtype: float64

In [64]:
from sklearn.model_selection import train_test_split

In [84]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.25,stratify=Y)

In [85]:
print(y_test.value_counts(normalize=True))
print(y_train.value_counts(normalize=True))
print(Y.value_counts(normalize=True))


0    0.73417
1    0.26583
Name: Churn, dtype: float64
0    0.734069
1    0.265931
Name: Churn, dtype: float64
0    0.734094
1    0.265906
Name: Churn, dtype: float64


In [86]:
from sklearn.svm import SVC

In [99]:
clf = SVC(gamma='auto',kernel="rbf",class_weight="balanced",degree=3)

In [100]:
clf.fit(X_train,y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight='balanced', coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [90]:
y_test.value_counts()

0    1287
1     466
Name: Churn, dtype: int64

In [101]:
y_pred=clf.predict(X_test)

In [92]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [102]:
confusion_matrix(y_pred,y_test)

array([[879,  75],
       [408, 391]])

In [103]:
print(classification_report(y_true=y_test,y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.92      0.68      0.78      1287
           1       0.49      0.84      0.62       466

    accuracy                           0.72      1753
   macro avg       0.71      0.76      0.70      1753
weighted avg       0.81      0.72      0.74      1753

